In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [6]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://www.pc.rs.gov.br/desaparecidos'
driver.get(website)

driver.maximize_window()

In [7]:
nomes = []
nascimentos = []
desaparecimentos = []
locais_desaparecimento = []

In [8]:
# Função para contar elementos
def contar_elementos(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath))

# Xpath para os cartões de desaparecidos
xpath_cards = "//div[@class='card-desaparecido']"

# Contagem inicial de elementos
contagem_inicial = contar_elementos(driver, xpath_cards)
scrolls_sem_novos_elementos = 0

# Loop para descer a página e coletar dados
while scrolls_sem_novos_elementos < 3:  # se nenhum novo cartão aparecer após 3 scrolls, assumimos que todos foram carregados
    # Scroll para o fim da página
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

    # Espera para carregar o conteúdo após o scroll
    time.sleep(5)  # Ajuste o tempo conforme necessário

    # Contagem após scroll
    contagem_apos_scroll = contar_elementos(driver, xpath_cards)

    # Verificação se novos elementos foram carregados
    if contagem_apos_scroll > contagem_inicial:
        contagem_inicial = contagem_apos_scroll
        scrolls_sem_novos_elementos = 0  # Reset do contador porque novos elementos apareceram
    else:
        scrolls_sem_novos_elementos += 1  # Incrementa o contador se não houver novos elementos

In [9]:
cards = driver.find_elements(By.XPATH, xpath_cards)
for card in cards:
    nome = card.find_element(By.XPATH, ".//h3[@class='card-desaparecido-nome']").text
    nomes.append(nome)
    dados = card.find_elements(By.XPATH, ".//p")
    nascimento = dados[0].text.split(":")[1].strip()
    nascimentos.append(nascimento)
    desaparecimento = dados[1].find_element(By.TAG_NAME, "strong").text
    desaparecimentos.append(desaparecimento)
    local_desap = dados[2].find_element(By.TAG_NAME, "strong").text
    locais_desaparecimento.append(local_desap)

In [ ]:
df_list = pd.DataFrame({'Nomes Desaparecidos': nomes, 'Data Nascimento': nascimentos, 'Data Desaparecimento': desaparecimentos, 'Local Desaparecimento': locais_desaparecimento})
df_list.to_csv('Desaparecidos RS.csv', index=False)